In [5]:
# FER2013 with TensorFlow / Keras (ResNet50 transfer learning)
# importations
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [6]:
# ---------- Config ----------
CSV_PATH = "fer2013.csv"               # path to your fer2013 file
IMG_SIZE = 224                         # ResNet expects larger images; we'll resize 48->224
BATCH_SIZE = 64
EPOCHS = 15
MODEL_SAVE_PATH = "fer_resnet50_tf"    # will save a SavedModel folder
NUM_WORKERS = tf.data.AUTOTUNE

In [7]:
# FER2013 label map
emotion_labels = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral"
}

In [8]:
# ---------- 1) Load CSV ----------
data = pd.read_csv(CSV_PATH)
print("Rows:", len(data))
print("Usage counts:\n", data['Usage'].value_counts())
print("Emotion distribution:\n", data['emotion'].value_counts())

Rows: 35887
Usage counts:
 Usage
Training       28709
PublicTest      3589
PrivateTest     3589
Name: count, dtype: int64
Emotion distribution:
 emotion
3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: count, dtype: int64


In [9]:
# Helper: convert "pixels" string -> (48,48,3) uint8 image (0-255)
def pixels_to_rgb_array(pixels_str):
    arr = np.fromstring(pixels_str, dtype=np.uint8, sep=' ')
    arr = arr.reshape(48, 48)                     # FER images are 48x48
    arr = np.stack([arr, arr, arr], axis=-1)     # convert to 3 channels
    return arr                                   # dtype uint8, range 0-255

In [10]:
# Build numpy arrays for each split (small memory usage; FER2013 is OK in RAM)
def build_split(df):
    imgs = np.stack([pixels_to_rgb_array(p) for p in df['pixels'].values])
    labels = df['emotion'].astype(np.int32).values
    return imgs, labels

train_df = data[data['Usage'] == 'Training'].reset_index(drop=True)
val_df   = data[data['Usage'] == 'PublicTest'].reset_index(drop=True)
test_df  = data[data['Usage'] == 'PrivateTest'].reset_index(drop=True)

X_train, y_train = build_split(train_df)
X_val,   y_val   = build_split(val_df)
X_test,  y_test  = build_split(test_df)

print("Train/Val/Test shapes:", X_train.shape, X_val.shape, X_test.shape)

Train/Val/Test shapes: (28709, 48, 48, 3) (3589, 48, 48, 3) (3589, 48, 48, 3)


In [11]:
# ---------- 2) tf.data pipelines ----------
# Preprocessing map: resize + preprocess_input (ResNet50)
def preprocess(img, label):
    # img: uint8 0-255
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))   # resize to model input
    img = preprocess_input(img)                        # ResNet50 preprocessing: RGB->BGR, mean subtraction or scaling
    return img, label

In [12]:
# Data augmentation for training
data_augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.08),
    layers.RandomZoom(0.08)
], name="data_augmentation")

In [13]:
def preprocess_train(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = data_augment(img)        # augmentation applied at train time
    img = preprocess_input(img)
    return img, label

In [14]:
# Create tf.data datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(10000).map(preprocess_train, num_parallel_calls=NUM_WORKERS).batch(BATCH_SIZE).prefetch(NUM_WORKERS)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).map(preprocess, num_parallel_calls=NUM_WORKERS).batch(BATCH_SIZE).prefetch(NUM_WORKERS)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(preprocess, num_parallel_calls=NUM_WORKERS).batch(BATCH_SIZE).prefetch(NUM_WORKERS)

In [15]:
# ---------- 3) Build model (transfer learning) ----------
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False   # start with frozen backbone

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(len(emotion_labels), activation="softmax")(x)

model = models.Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 35s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,114,055 (91.99 MB)

 Trainable params: 526,343 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [16]:
# 4 Callbacks
cb = [
    callbacks.ModelCheckpoint("best_fer_resnet50.h5", save_best_only=True, monitor="val_accuracy", verbose=1),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
    callbacks.EarlyStopping(monitor="val_accuracy", patience=6, restore_best_weights=True, verbose=1)
]

In [ ]:
#5 Train
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=cb
)

# Optional: fine-tune: unfreeze some layers if you want more performance
# base_model.trainable = True
# for layer in base_model.layers[:-10]:
#     layer.trainable = False
# recompile with lower lr and continue training

Epoch 1/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.3146 - loss: 1.7705
Epoch 1: val_accuracy improved from None to 0.46085, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 2469s 5s/step - accuracy: 0.3720 - loss: 1.6323 - val_accuracy: 0.4609 - val_loss: 1.4251 - learning_rate: 1.0000e-04
Epoch 2/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4366 - loss: 1.4811
Epoch 2: val_accuracy improved from 0.46085 to 0.48593, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 2028s 5s/step - accuracy: 0.4474 - loss: 1.4555 - val_accuracy: 0.4859 - val_loss: 1.3613 - learning_rate: 1.0000e-04
Epoch 3/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4591 - loss: 1.4175
Epoch 3: val_accuracy improved from 0.48593 to 0.50571, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 2053s 5s/step - accuracy: 0.4683 - loss: 1.3973 - val_accuracy: 0.5057 - val_loss: 1.3199 - learning_rate: 1.0000e-04
Epoch 4/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4814 - loss: 1.3689
Epoch 4: val_accuracy improved from 0.50571 to 0.51323, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 2002s 4s/step - accuracy: 0.4879 - loss: 1.3592 - val_accuracy: 0.5132 - val_loss: 1.2938 - learning_rate: 1.0000e-04
Epoch 5/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4908 - loss: 1.3363
Epoch 5: val_accuracy improved from 0.51323 to 0.52076, saving model to best_fer_resnet50.h5


...

449/449 ━━━━━━━━━━━━━━━━━━━━ 2390s 5s/step - accuracy: 0.5097 - loss: 1.2996 - val_accuracy: 0.5235 - val_loss: 1.2537 - learning_rate: 1.0000e-04
Epoch 7/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5155 - loss: 1.2927
Epoch 7: val_accuracy improved from 0.52354 to 0.53246, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 2009s 4s/step - accuracy: 0.5208 - loss: 1.2827 - val_accuracy: 0.5325 - val_loss: 1.2460 - learning_rate: 1.0000e-04
Epoch 8/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5220 - loss: 1.2688
Epoch 8: val_accuracy improved from 0.53246 to 0.54138, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 1987s 4s/step - accuracy: 0.5232 - loss: 1.2662 - val_accuracy: 0.5414 - val_loss: 1.2271 - learning_rate: 1.0000e-04
Epoch 9/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5270 - loss: 1.2639
Epoch 9: val_accuracy did not improve from 0.54138
449/449 ━━━━━━━━━━━━━━━━━━━━ 1979s 4s/step - accuracy: 0.5319 - loss: 1.2504 - val_accuracy: 0.5350 - val_loss: 1.2254 - learning_rate: 1.0000e-04
Epoch 10/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5378 - loss: 1.2315
Epoch 10: val_accuracy improved from 0.54138 to 0.54974, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 1984s 4s/step - accuracy: 0.5366 - loss: 1.2327 - val_accuracy: 0.5497 - val_loss: 1.2151 - learning_rate: 1.0000e-04
Epoch 11/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5378 - loss: 1.2297
Epoch 11: val_accuracy did not improve from 0.54974
449/449 ━━━━━━━━━━━━━━━━━━━━ 2409s 5s/step - accuracy: 0.5378 - loss: 1.2283 - val_accuracy: 0.5478 - val_loss: 1.2028 - learning_rate: 1.0000e-04
Epoch 12/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5496 - loss: 1.2116
Epoch 12: val_accuracy improved from 0.54974 to 0.55475, saving model to best_fer_resnet50.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 2145s 5s/step - accuracy: 0.5509 - loss: 1.2049 - val_accuracy: 0.5548 - val_loss: 1.1908 - learning_rate: 1.0000e-04
Epoch 13/15
 15/449 ━━━━━━━━━━━━━━━━━━━━ 34:37 5s/step - accuracy: 0.5865 - loss: 1.2142